<a href="https://colab.research.google.com/github/FabioPojo1987/FabioPojo1987-MVP_2_DATA_SCIENCE_PUC_RIO/blob/main/MVP_2_DATA_SCIENCE_PUC_Rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto de MVP da 2ª Sprint do curso de Ciência de Dados na PUC-Rio (por Fabio Pojo de Almeida).**


Este projeto se trata na construção de um modelo de aprendizado de máquina ("machine learning") para criar insights sobre como ocorre a variação de dados dos valores das ações do Google no Mercado de Ações.

Onde foi utilizado um recorte temporal compreendido entre os anos de 2010 e 2023

Este projeto está baseado em um modelo de "Rede Neural Recorrente" e está usando de referência os dados presentes nas planilhas csv

Para isso, foi decidido um exemplo onde esses dados foram separados em duas bases em 2 arquivos diferentes: 1- base de treino e 2- teste (a faixa da base de teste tem como referência um período de 3 semanas, ou 21 dias)

- A base de dados utilizada para os valores das ações do Google (Alphabet.inc.) foi obtida através do Kaggle em: https://www.kaggle.com/datasets/alirezajavid1999/google-stock-2010-2023/download?datasetVersionNumber=1


Campos do dataset :

1.   Date
2.   Open - Valor das ações na Abertura
3.   High - Valor mais alto do dia ("Pico")
4.   Low - Valor mais baixo ("vale")
5.   Volume - Volume a ser Negociado
6.   High - Close : Valor (a maior) no momento de fechamento


Objetivo - Usar os dados de treino para prever os valores que estão nos dados de teste, o objetivo é comparar a previsão do modelo com o que de fato aconteceu.





In [ ]:
# Importar as bibliotecas
import numpy as np # vai ser usado pra criar os arrays
import pandas as pd
import tensorflow as tf

import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt # para gerar graficos
import plotly.graph_objects as go

from tensorflow import keras

from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

In [ ]:
url_dados = 'https://github.com/FabioPojo1987/FabioPojo1987-MVP_2_DATA_SCIENCE_PUC_RIO/blob/main/Google_Stock_Train%20(2010-2022).csv'

In [ ]:
#importar os dados
dataset_train = pd.read_csv(url_dados)
training_set = dataset_train.iloc[:, 1:2].values # separar apenas os atributos
# que usaremos no modelo

In [ ]:
google_training_complete = pd.read_csv("/kaggle/input/google-stock-2010-2023/Google_Stock_Train (2010-2023).csv")

# Convert 'Date' column to datetime format
google_training_complete['Date'] = pd.to_datetime(google_training_complete['Date'])

google_training_complete.head(10)

In [ ]:
# Desenhando gráfico em linhas dos valores de ação do Google no decorrer do tempo
fig1 = px.line(google_training_complete, x='Date', y='Close', title='Valores de ação do Google no decorrer do tempo')
fig1.show()

In [ ]:
# Fazendo o volume do valor diário das ações
fig2 = px.scatter(google_training_complete, x='Date', y='Volume', title='Volume do Valor Diário das ações')
fig2.show()

In [ ]:
#Transformação
from sklearn.preprocessing import MinMaxScaler # para colocar os valores numa mesma escala
sc = MinMaxScaler(feature_range = (0, 1)) # a escala de normalização deve ser entre 0 e 1
training_set_scaled = sc.fit_transform(training_set) # Escala para aplicar os dados no ato da normalização

In [ ]:
#Estruturas de dados com 60 intervalos de tempo, devemos usar
# sempre 60 dias para poder fazer previsão da proxima saida
X_train = [] # Criando a lista do Python para coletar dados desses 60 dias
y_train = [] # Definindo a lista de previsão para o proximo dia
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Nesta etapa, serão organizados os dados de acordo com os formatos que são exigidos pelo Keras.
#E,para que o Keras possa reconhecer, precisa estar conforme os seguintes parâmetros
# batch_size, timesteps (para definição dos intervalos de treinamento, e quantitativos de indicadores a serem usados)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
#Para usar o Keras, é necessário importar as bibliotecas do Keras abaixo:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Nesta parte, devemos colocar os parâmetros necessários para podermos inicializar a rede neural
regressor = Sequential()
#A seguir, devemos criar as camadas
#1
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
#2
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
#3
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
#4
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
# camada de saída (output)
regressor.add(Dense(units = 1))

In [ ]:
# Após definir a rede neural, devemos compilar a rede
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# Em seguida, definiremos os parâmetros de treinamento
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
url_dados2 = 'https://github.com/FabioPojo1987/FabioPojo1987-MVP_2_DATA_SCIENCE_PUC_RIO/blob/main/Google_Stock_Train%20(2010-2022).csv'

In [ ]:
#Definir o Dataset para obter os dados reais
dataset_test = pd.read_csv(url_dados2)
real_stock_price = dataset_test.iloc[:, 1:2].values # Esse comando é para pegar apenas a coluna de saída (Open)

In [ ]:
#Para obter os dados de previsão
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1) # deve usar esse comando para definir a variável passando entre -1 e 1 como parâmetro
inputs = sc.transform(inputs) # este comando serve para gerar a normalização dos dados
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Gerando a Visualizção dos Resultados
plt.plot(real_stock_price, color = 'red', label = 'Valores reais das Ações do Google')
plt.plot(predicted_stock_price, color = 'blue', label = 'Valores previstos das Ações do Google')
plt.title('Previsão de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações do Google')
plt.legend()
plt.show()

A partir deste trabalho, foi possível observar que o sistema de previsão até que conseguiu obter uma tendência bem interessante, especialmente sendo para pessoas que lidam com o mercado de ações na Bolsa de Valores.

Contudo, ainda assim o algoritmo não conseguiu acertar as tendências. Errou em parte.

Pois, utilizar esse tipo de metodologia (para tentar prever dados de ações financeiras) é algo bastante complexo e demanda muito tempo e poder de processamento para ter mais eficiência e precisão.

Fora que, devido ao fato de se lidar com informações de risco, não é aconselhavel de se basear os investimentos de capital em modelos de simulação como esse.

Porém, pode-se utilizar esse tipo de metodologia pra estudar e aprender novas possibilidades de previsões e de tendência.